#### Underactuation in OPC
1. For any general robotic system, the relationship between the end-effector forces $F$ and the actuator torques $\Gamma$ is such that $\tau = J^T F$
2. For a redundant system, there are infinitely many posible joint torques that produces the same $F$. In OPC, the selection is based on minimizing the acceleration energy $\frac{1}{2} \Gamma^T M^{-1} \Gamma$ or equivalently the kinetic energy $\frac{1}{2} \dot q^T M^{-1} \dot q$, resulting in the dynamically consistent jacobian $\bar J = M^{-1} J^T \Lambda$. Thus, $F = \bar J^T \Gamma$ 
3. In the case of an underactuated system, $\Gamma$ may contain zero elements representing the joints that are uncontrolled. The mapping between the actuation torques $\Gamma_a$ to the full $\Gamma$ can be represented as $\Gamma_a = S \Gamma$ where $S \in \mathbb{R}^{n_a \times n}$. The rows of $S$ contains all zeros with a single $1$ to select a joint in $\Gamma$ to be controlled.
4. With $F = \bar J^T \Gamma$, we get $F = \bar J^T S^T \Gamma_a$, i.e. the mapping between actuated joints to the resulting end effector forces. Mapping back to joint space, we thus have $\Gamma_a = \overline{\bar {J}^T S^T} F + N^T \Gamma_{a,null}$ where $N^T = I - \bar J_a^T J_a^T$ and $J_a^T = \overline{\bar {J}^T S^T}$

#### Task space controller
1. The vanilla operational space control for a redundant robot is designed as follows: $\Gamma = J_t^T F_t + J_{p|t}^T F_{p|t}^T$ where $p|t$ indicates that the posture task is dynamically consistent with the main tasks, i.e. it does not induce accelerations in the main task space.
2. Again, we can further simplify the controller where we compensate coriolis and gravity forces in joint space, resulting in the closed loop dynamics of $M \ddot q + C \dot q + g = \Gamma^\prime + \hat{C} \dot q + \hat {g}$. With perfect estimates, we are left with $M \ddot q = \Gamma^\prime$
3. The controller is designed such that $\Gamma^\prime = J_t^T F_t + J_{p|t}^T F_{p|t}^T$. For subsequent discussion, we will drop the $^\prime$ for readability. Let's assume that our we are not controlling our nullspace for now, i.e. $F_{p|t} = 0$. The resulting closed loop system is $M \ddot q = J_t^T \Lambda_t \alpha_t$. Pre-multiplying with $J_t M^{-1}$ yields $\ddot x - \dot J \dot q = \alpha_t$. Clearly, using $\alpha_t = \ddot x_d + K_p e + K_d \dot e - \dot J \dot q$ results in a linear second order error dynamics.

#### Additive task space controller
1. Now, supposedly we include an additive controller, i.e. $\Gamma = J_{t,1}^T F_{t,1} + J_{t,2}^T F_{t,2}$. For the first controller, we are spanning all the joints to control the 6 Dof end effector task. For the second controller, we will only be selecting the distal 5 joints. Thus, the second controller will be designed with the structure $\Gamma_{t,2} = J_{t,2}^T F_{t,2} = S^T \Gamma_a$
2. Our controller is now $\Gamma = J^T \Lambda \alpha_{1} + S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2}$. The closed loop dynamics is such that $M \ddot q = J^T \Lambda \alpha_{1} + S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2}$. Again, we select $\alpha_1 = \ddot x_d + K_{p,1} e + K_{d,1} \dot e - \dot J \dot q$, premultiplying with $J M^{-1}$, we arrive at
\begin{align}
J \ddot q &= \alpha_{1} + J M^{-1} S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2} \\
\ddot x - \dot J \dot q &= \ddot x_d + K_p e + K_d \dot e - \dot J \dot q + J M^{-1} S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2} \\
0 &= \ddot e + K_{p,1} e + K_{d,1} \dot e + J M^{-1} S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2}
\end{align}
3. Lets select $\alpha_2 = K_{p,2} e + K_{d,2} \dot e$ and denote the last term in the equation above as $\beta = J M^{-1} S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2}$. Lets expose the properties of $\beta$
\begin{align}
    \beta &= J M^{-1} S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2} \\
          &= \Lambda^{-1} \Lambda J M^{-1} S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2} \\
          &= \Lambda^{-1} \bar J^T S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2} \\
\end{align}
Now, using Gauss's principlem we derive $\overline{\bar {J}^T S^T} = W^{-1} (\bar J^T S^T)^T [\bar J^T S^T W^{-1} (\bar J^T S^T)^T]^{+}$ with $W = SA^{-1}S^T$ (https://sci-hub.se/10.1109/ROBOT.2000.846414). Since $\bar J^T S^T W^{-1} (\bar J^T S^T)^T \in \mathbb{R}^{6 \times 6}$ but has a rank of $n_a < 6$ (due to underactuation), we will be using the moore-penrose pseudoinverse, as denoted by $(\cdot)^{+}$
4. Expanding out $\overline{\bar {J}^T S^T}$ in the term $\bar J^T S^T \overline{\bar {J}^T S^T}$ yields
\begin{align}
    \bar J^T S^T \overline{\bar {J}^T S^T} &= \bar J^T S^T W^{-1} (\bar J^T S^T)^T [\bar J^T S^T W^{-1} (\bar J^T S^T)^T]^{+} \\
    &= \bar J^T S^T W^{-1} S^T \bar J [\bar J^T S^T W^{-1} S^T \bar J]^{+} \\
    &= \bar J^T W_s^+ \bar J [\bar J^T W_s^+ \bar J]^{+}
\end{align}
where $W_s^+ \triangleq S^T W^{-1} S^T$. Clearly, since $W \in \mathbb{S}^++$ and that $S$ is the selection matrix, $W_s^+ \in \mathbb{S}^+$. Note that $W \in \mathbb{R}^{n_a \times n_a}$ while $W_s^+ \in \mathbb{R}^{n \times n}$. This further indicates that $\bar J^T S^T \overline{\bar {J}^T S^T} \in \mathbb{S}^+$

5. Going back to $\beta = \Lambda^{-1} \bar J^T S^T \overline{\bar {J}^T S^T} \Lambda \alpha_{2}$, we let $T \triangleq \Lambda^{-1} \bar J^T S^T \overline{\bar {J}^T S^T} \Lambda \in \mathbb{S}^+$. Clearly, due to the positive semi-definiteness of $T$, the angle between the vector $\alpha_2$ and $\beta$ will be orthogonal ($T = 0$) or acute ($T > 0$). Now, let us take $\beta$ as $K_{p,2} e + K_{d,2} \dot e$. This can be physically interpreted that we are mounting additional springs and dampers in parallel to the existing feedback control from $\alpha_1$. This is equivalent to $T = I$ and $\alpha_2 = K_{p,2} e + K_{d,2} \dot e$. 

6. Due to underactuation, $\alpha_2$ will be remapped by $T$ to produce components that not identical to $\beta$. To better understand this remapping, we can perform orthogonal diagonalization on $T$, i.e. $T = Q^T \Sigma Q$. $Q$ is a orthogonal matrix recovered from the Gram-Schmidt process and can be interpreted as a rotation operator on $\alpha_2$. $T\alpha_2$ first rotates $\alpha_2$ by $Q$. Next, $\Sigma = diag[1,\dots,1,0,\dots,0]$ will then perform element-wise pass-through or surpression of $Q \alpha_2$. Finally, $Q^T$ will then rotate the result back to the original bases.
7. If we take both $\beta$ and $\alpha_2$ as $K_{p,2} e + K_{d,2} \dot e$, the resulting deviation is such that $e_T = [I - \Lambda^{-1} \bar J^T S^T \overline{\bar {J}^T S^T} \Lambda] [K_{p,2} e + K_{d,2} \dot e]$. Another althernative is that, instead of specfying $\alpha_2 = K_{p,2} e + K_{d,2} \dot e$, we choose $\alpha_2$ to preferentially minimize errors in the subspace of $\beta$ using $\alpha_2 = T^\#_A \beta = T^\#_A [K_{p,2} e + K_{d,2} \dot e]$ where $A$ is the desired weighting matrix. **Check this method later**

#### Analysis on the nullspaces
1. Thus far, our focus is on the task space, how then does the nullspaces from both controllers affect each other? To

#### Improvements
1. How then do we introduce the additive controller? Should it be activated all the time? If not, how should we design a shaped projector to ramp-in the additive torques?